In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
from os import listdir
from os.path import isfile, join, abspath
import pandas as pd
import xlwings as xw
import numpy as np
import quandl
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from TFANN import ANNR
import seaborn as sns
from scipy import stats
import os
%matplotlib inline 

C:\Users\clare\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# This code gets the file names of all CSVs of interests
filepath = str(input("Entire file path of CSVs: " + "\n"))
def computer(filepath):
    if filepath[2] == '\\':
        filep = filepath[2:]
        path_for_windows = filep
        path = path_for_windows.replace('\\','/')
        return path
    else:
        return filepath
p = computer(filepath)
onlyfiles = [f for f in listdir(p) if isfile(join(p, f))]
final = []
for i in onlyfiles:
    final.append(p +"/"+ i)
print(final) #C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Dat

Entire file path of CSVs: 
C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Dat
['/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_01_May_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Apr_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Jul_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_02_Jun_2019.csv', '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_t

In [3]:
# Gets the currencies involved
dcc = pd.read_csv('datcommodcurr.csv',skiprows=1)#, header = None, sheet_name = 'Reference_Sheet')
dcc.index = dcc["Unnamed: 0"]
# dcc['AAP']['3/6/18']
listcurr = []
for i in dcc.columns:
    if "/" in i:
        listcurr.append(i)
listcurr
listc=[]
for i in listcurr:
    listc.append(dcc[i])
dff = pd.DataFrame(listc)
dafr = dff.T
dafr["Date"] = dafr.index
dafr.index = list(range(len(dafr)))
indx = list(range(len(dafr)))
wknds = []
wkdys = []
for i in range(len(dafr)):
    try:
        if dafr.iloc[i]['AUD/CAD'] == dafr.iloc[i+1]['AUD/CAD'] and dafr.iloc[i+1]['AUD/CAD'] == dafr.iloc[i+2]['AUD/CAD']:
            wknds.append(i+1)
            wknds.append(i+2)
    except:
        continue
for i in wknds:
    if i in indx:
        indx.remove(i)
dafr
finaldf=dafr[np.in1d(np.arange(len(dafr)), indx)]
finaldf.index = finaldf["Date"]
# del finaldf['Date']
finaldf # gets df without the weekends

,AUD/CAD,AUD/CHF,AUD/EUR,AUD/GBP,AUD/JPY,AUD/NZD,AUD/USD,BTC/EUR,CAD/AUD,CAD/CHF,...,USD/NOK,USD/PLN,USD/RUB,USD/SEK,USD/SGD,USD/THB,USD/TRY,USD/UAH,USD/ZAR,Date
Date,,,,,,,,,,,,,,,,,,,,,
3/16/18,1.0100,0.7344,0.6275,0.5532,81.76,1.0682,0.7715,6935.5060,0.9891,0.7268,...,7.7131,3.4324,57.5056,8.1914,1.3173,31.23,3.9186,26.4000,11.9695,3/16/18
3/19/18,1.0090,0.7338,0.6255,0.5502,81.84,1.0649,0.7717,6828.2400,0.9905,0.7270,...,7.7219,3.4238,57.7662,8.1680,1.3168,31.18,3.9338,26.3600,12.0095,3/19/18
3/20/18,1.0041,0.7346,0.6274,0.5488,81.84,1.0689,0.7682,7280.8315,0.9956,0.7316,...,7.7470,3.4556,57.5563,8.2202,1.3187,31.18,3.9329,26.3200,11.9585,3/20/18
3/21/18,1.0013,0.7371,0.6291,0.5489,82.32,1.0740,0.7765,7207.0990,0.9981,0.7359,...,7.6955,3.4275,56.8620,8.1821,1.3122,31.15,3.9059,26.2800,11.8200,3/21/18
3/22/18,0.9949,0.7298,0.6250,0.5455,80.96,1.0670,0.7691,6988.7840,1.0042,0.7333,...,7.7489,3.4327,57.2260,8.2550,1.3167,31.30,3.9324,26.2600,11.8367,3/22/18
3/23/18,0.9921,0.7288,0.6229,0.5445,80.59,1.0631,0.7696,6975.0303,1.0073,0.7344,...,7.7543,3.4229,57.2234,8.2457,1.3152,31.18,3.9814,26.2500,11.7353,3/23/18
3/26/18,0.9945,0.7323,0.6225,0.5443,81.61,1.0608,0.7746,6339.3070,1.0046,0.7361,...,7.6810,3.3918,57.1970,8.1893,1.3085,31.12,3.9647,26.2650,11.6174,3/26/18
3/27/18,0.9889,0.7267,0.6189,0.5422,80.83,1.0570,0.7677,6444.9820,1.0104,0.7347,...,7.7175,3.3966,57.3560,8.2296,1.3096,31.21,3.9803,26.2900,11.6655,3/27/18
3/28/18,0.9896,0.7326,0.6222,0.5440,81.86,1.0624,0.7659,6417.7173,1.0095,0.7400,...,7.8621,3.4146,57.7388,8.3495,1.3114,31.27,4.0076,26.4400,11.7678,3/28/18


In [4]:
# Helper function 
def fivemax(ccsv):
    list1 = [abs(number) for number in list(ccsv['Signal'])]
    final_list = [] 
    for i in range(0, 5):  
        max1 = 0
        for j in range(len(list1)):      
            if list1[j] > max1: 
                max1 = list1[j]; 
        list1.remove(max1); 
        final_list.append(str(max1)) 
    return(final_list) 
csv1 = '/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_09_Apr_2019.csv'
import itertools
def gettopfivedf (csv11):
    csv = pd.read_csv(csv11)
    for1 = csv[0:10]
    for2 = csv[10:20]
    for3 = csv[20:30]
    for4 = csv[30:40]
    for5 = csv[40:50]
    for6 = csv[50:60]
    for2.index= list(range(len(for2)))
    for3.index= list(range(len(for3)))
    for4.index= list(range(len(for4)))
    for5.index= list(range(len(for5)))
    for6.index= list(range(len(for6)))
    dlist = [for1,for2,for3,for4,for5,for6]
    futdf=[]
    for y in dlist:
        for r in fivemax(y):
            for i in range(len(y)):
                if r in str(y["Signal"][i]):
                    futdf.append([y['Ticker'][i],y['Time Horizon'][i],y['Predictability'][i],y['Signal'][i]])
    gf = pd.DataFrame(futdf, columns = ["Currency", "Time Horizon", "Predictability", "Signal"])
    dateee=(csv11[-15:].replace("_","/").replace(".csv","").replace("May","05").replace("Mar","03").replace("Apr","04").replace("Jun","06").replace("Jul","07").replace("Aug","08"))
    datetimeobject = datetime.strptime(dateee,'%d/%m/%Y') # to change from dd/mm/yyyy to a datetime object
    final_date = datetimeobject.strftime('%d/%m/%Y')# to change to mm/dd/yyyy
    L = list(itertools.repeat(final_date, len(gf)))
    gf["Date"] = L
    return(gf)
listofdfs=[]
for u in final:
    fin = gettopfivedf(u)
    listofdfs.append(fin)
df_top_5 = pd.concat(listofdfs)
df_top_5.index = list(range(len(df_top_5)))
df_top_5 = df_top_5.applymap(str)
df_top_5

,Currency,Time Horizon,Predictability,Signal,Date
0,RUB/GBP,1d,0.15,-10.775369760345272,01/05/2019
1,AUD/GBP,1d,0.16,-8.27926040675232,01/05/2019
2,GBP/AUD,1d,0.14,7.712282818873827,01/05/2019
3,USD/IDR,1d,0.12,7.2774579093211225,01/05/2019
4,EUR/RUB,1d,0.13,6.607806002460362,01/05/2019
5,RUB/GBP,2d,0.18,-8.979474800287727,01/05/2019
6,AUD/GBP,2d,0.18,-7.819301495266081,01/05/2019
7,GBP/AUD,2d,0.16,7.306373196827834,01/05/2019
8,USD/IDR,2d,0.14,7.2774579093211225,01/05/2019
9,EUR/RUB,2d,0.15,6.19481812730659,01/05/2019


In [34]:
DATA=[]
specific =[]
for u in final:
# u = 'C:/Users/clare/OneDrive/Documents/Internships and work/IKNOWFIRST/IKF WORK/project/Portfolio/Dat/IKForecast_best_short_term_currencies_top30_pred_top10_sig_filter_flat_01_May_2019.csv' 
    actual = u[-15:]
    datee = (u[-15:].replace("_","/").replace(".csv","").replace("2019","19").replace("May","5").replace("Mar","3").replace("Apr","4").replace("Jun","6").replace("Jul","7").replace("Aug","8")
)
    date = datee.replace("0","")
    dateee=(u[-15:].replace("_","/").replace(".csv","").replace("May","05").replace("Mar","03").replace("Apr","04").replace("Jun","06").replace("Jul","07").replace("Aug","08")
)
    df = pd.read_csv(u)
    for i in range (len(df)):
        for curr in listcurr:
            if df['Ticker'][i] == curr:
                DATA.append([date, df['Ticker'][i],df['Time Horizon'][i],df['Predictability'][i],df['Signal'][i]])
                specific.append([dateee, df['Ticker'][i],df['Time Horizon'][i],df['Predictability'][i],df['Signal'][i]])         
ee = pd.DataFrame(specific)
ee.columns=(["Date","Currency","Time Horizon","Predictability","Signal"])
ee.index = ee["Date"]
t = pd.DataFrame(DATA)
t.columns=(["Date","Currency","Time Horizon","Predictability","Signal"])
t.index = t["Date"]
ee # dd/mm/yyyy df
for i in range (len(ee)):
    if ee["Currency"][i] == "USD/MXN":
        print(ee["Time Horizon"][i])

1d


In [6]:
ID = pd.read_excel("Investing.com ID.xlsx")
ID
for t in range(len(ID)):
    if ID["Currency"][t] == 'USD/ZAR':
        curr_id = ID["CurrID"][t]
        smlID = ID["smlID"][t]
        print(curr_id)
        print(smlID)

17
106698


In [7]:
def investing(i,std,etd):
    ID = pd.read_excel("Investing.com ID.xlsx")
    for t in range(len(ID)):
        if ID["Currency"][t] == i:
            headers = {
                "Host": "www.investing.com",
                "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:67.0) Gecko/20100101 Firefox/67.0",
                "Accept": "text/plain, */*; q=0.01",
                "Accept-Language": "en-US,en;q=0.5",
                "Accept-Encoding": "gzip, deflate, br",
                "Referer": "https://www.investing.com/currencies/"+i.replace("/","-").lower()+"-historical-data",
                "Content-Type": "application/x-www-form-urlencoded",
                "X-Requested-With": "XMLHttpRequest",
                "Content-Length": "173"
            }
            body = {
                "curr_id": ID["CurrID"][t],
                "smlID": ID["smlID"][t],
                "header": i+"+Historical+Data",
                "st_date": std,
                "end_date": etd,
                "interval_sec": "Daily",
                "sort_col": "date",
                "sort_ord": "DESC",
                "action": "historical_data"
            }
            url = "https://www.investing.com/instruments/HistoricalDataAjax"
            re = requests.post(url, data=body, headers=headers)
            soup = BeautifulSoup(re.content, 'html.parser')
            table_rows = soup.findAll('tr')
            dates = []
            price = []
            open_price = []
            high = []
            low_price = []
            percentage_changes = []
            for tr in table_rows:
                metadata = tr.findAll('td', {'data-real-value': True})
                try:
                    dates.append(metadata[0].text.replace(",","/").replace("2019","19").replace("Jan","1/").replace("Feb","2/").replace("Mar","3/").replace("Apr","4/").replace("May","5/").replace("Jun","6/").replace("Jul","7/").replace("Aug","8/").replace("Sep","9/").replace("Oct","10/").replace("Nov","11/").replace("Dec","12/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9").replace(" ",""))
                    price.append(metadata[1].text)
                    open_price.append(metadata[2].text)
                    high.append(metadata[3].text)
                    low_price.append(metadata[4].text)
                except:
                    continue
                try:
                    percentage_changes.append(tr.findAll('td', {'class': 'redFont'})[1].text)
                except:
                    percentage_changes.append(tr.findAll('td', {'class': 'greenFont'})[1].text)
            tot = []
            tot.append(dates)#.replace(",","/").replace("2019","19").replace("Jan","1/").replace("Feb","2/").replace("Mar","3/").replace("Apr","4/").replace("May","5/").replace("Jun","6/").replace("Jul","7/").replace("Aug","8/").replace("Sep","9/").replace("Oct","10/").replace("Nov","11/").replace("Dec","12/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
            tot.append(open_price)
#             tot.append(price)
#             tot.append(high)
#             tot.append(low_price)
#             tot.append(percentage_changes)
            df1 = pd.DataFrame(tot)
            df = df1.T
#             df.columns=["Dates", "Close","Open","High","Low","% Change"]
            df.columns=["Dates", i]
            df.index= df["Dates"]
            del df["Dates"]
            return(df) 
investing("AUD/CAD", "03/27/2019", "07/18/2019")

,AUD/CAD
Dates,
7/18/19,0.9147
7/17/19,0.9179
7/16/19,0.9186
7/15/19,0.9124
7/14/19,0.9146
7/12/19,0.9119
7/11/19,0.9103
7/10/19,0.9097
7/9/19,0.9128


In [9]:
# ii = 'USD/NOK'
std1 = "03/26/2019"
etd1 = "07/28/2019"
# investing(ii,std1,etd1)
comb = []
for ii in listcurr:
    comb.append(investing(ii,std1,etd1) )
combined = pd.concat(comb,axis=1,sort=False)
combined # Dataframe of open prices

,AUD/CAD,AUD/CHF,AUD/EUR,AUD/GBP,AUD/JPY,AUD/NZD,AUD/USD,BTC/EUR,CAD/AUD,CAD/CHF,...,USD/MYR,USD/NOK,USD/PLN,USD/RUB,USD/SEK,USD/SGD,USD/THB,USD/TRY,USD/UAH,USD/ZAR
7/28/19,0.9107,0.6873,0.6218,0.5586,75.18,1.0425,0.6918,"8,521.0",1.0980,0.7547,...,NaN,8.7177,3.8410,NaN,9.5010,1.3698,30.885,5.6718,NaN,14.2963
7/26/19,0.9150,0.6889,0.6238,0.5579,75.53,1.0431,0.6951,"8,881.6",1.0926,0.7526,...,4.1145,8.6735,3.8192,63.2336,9.4513,1.3674,30.970,5.7025,25.4827,14.0818
7/25/19,0.9170,0.6873,0.6262,0.5589,75.50,1.0406,0.6977,"8,770.4",1.0909,0.7494,...,4.1185,8.6463,3.8178,63.2936,9.4322,1.3647,30.920,5.7086,25.5072,13.8831
7/24/19,0.9202,0.6902,0.6280,0.5631,75.82,1.0449,0.7006,"8,838.6",1.0870,0.7502,...,4.1185,8.6781,3.8153,63.1609,9.4697,1.3655,30.915,5.7209,25.5615,13.9254
7/23/19,0.9230,0.6909,0.6276,0.5639,75.89,1.0409,0.7035,"9,222.8",1.0834,0.7485,...,4.1095,8.6068,3.7883,63.0960,9.4156,1.3607,30.835,5.6795,25.5900,13.8635
7/22/19,0.9200,0.6912,0.6277,0.5631,75.83,1.0408,0.7041,"9,444.6",1.0858,0.7515,...,4.1140,8.5802,3.7887,63.0295,9.3841,1.3598,30.840,5.6403,25.7650,13.9460
7/21/19,0.9204,0.6916,0.6280,0.5633,75.87,1.0409,0.7045,"9,602.3",1.0865,0.7515,...,NaN,8.5733,3.7896,NaN,9.3841,1.3602,30.835,5.6525,NaN,13.9338
7/19/19,0.9217,0.6944,0.6275,0.5637,75.92,1.0433,0.7076,"9,450.0",1.0850,0.7532,...,4.1115,8.5373,3.7725,62.8087,9.3107,1.3569,30.820,5.6176,25.9150,13.8425
7/18/19,0.9147,0.6922,0.6246,0.5638,75.67,1.0412,0.7010,"8,643.3",1.0932,0.7568,...,4.1140,8.5905,3.7973,62.9353,9.3676,1.3611,30.900,5.6859,25.9094,14.0163
7/17/19,0.9179,0.6928,0.6254,0.5652,75.91,1.0464,0.7013,"8,396.8",1.0896,0.7548,...,4.1150,8.5747,3.7965,63.0391,9.3896,1.3583,30.945,5.7134,26.1877,13.9488


In [11]:
# for i in (combined["USD/RUB"].index):
#     if combined["USD/RUB"][i] == None:
#         print(combined["USD/RUB"][i])
#
combined["USD/RUB"]["5/2/19"]

'64.6373'

In [39]:
# helper function
import datetime
def addbusdays(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += datetime.timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date
def resultsopen (pair,Hor): 
    # this function creates a table and chart for returns and signal
    cp = pair
    TimeHorizon = Hor
    dayreturn = int([char for char in TimeHorizon][0]) +1
    currpair = []
    for r in range(len(ee)):
        if ee["Currency"][r] == cp and ee["Time Horizon"][r] == TimeHorizon:
            currpair.append([ee["Date"][r], ee["Signal"][r]])# to change to mm/dd/yyyy
    g = pd.DataFrame(currpair)
    g.columns = ["Date", "Signal"]
    g.index = g["Date"]
    forcst = []
    retdate = []
    of =[]
    for i in g["Date"]:
        q = datetime.datetime.strptime(i,'%d/%m/%Y') # remove time
        of.append(q.strftime("%m/%d/%Y"))
        forcst.append(q.strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
        retdate.append(addbusdays(q, dayreturn).strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
    g["old format"] = of
    g["Forecast Date"]=forcst
    g["End of return"] = retdate
    dattes = list(finaldf.index)
    newdlist=[]
    for i in range(len(g)):
        if g["Forecast Date"][i] not in dattes:
            newf = (datetime.datetime.strptime(g["old format"][i],"%m/%d/%Y"))-timedelta(days=2)
            newdlist.append(newf.strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
        else:
            newdlist.append(g["Forecast Date"][i])
    g["Start of return"]=newdlist
    returns=[]
    for i in range(len(g)):
        try:
            returns.append(((float(combined[cp][g["End of return"][i]]) - dcc[cp][g["Start of return"][i]])/dcc[cp][g["Start of return"][i]])*100)
        except:
            continue
    g["Returns"]= returns
    alif = []
    alll = []
    for i in range(len(g)):
        if (g["Signal"][i] > 0 and g["Returns"][i] > 0) or (g["Signal"][i] < 0 and g["Returns"][i] < 0):
            alll.append(abs(g["Returns"][i]))
        else:
            if g["Returns"][i] < 0:
                alif.append(g["Returns"][i])
            else:
                alif.append(-1*g["Returns"][i])
    alif = [x for x in alif if str(x) != 'nan']
    alll = [x for x in alll if str(x) != 'nan'] 
    
    if len(alif) > 0:
        average_losses= sum(alif)/len(alif)
    else:
        average_losses= 0
    if len(alll) >0:
        average_gains= sum(alll)/len(alll)
    else:
        average_gains= 0
    average_returns = average_gains + average_losses #Average returns just by following signal per currency pair
    hit_ratio = (len(alll)/len(g))*100
    untouched = []
    for i in g["Returns"]:
        untouched.append(i)
    untouched = [x for x in untouched if str(x) != 'nan']
    real_returns = sum(untouched)/len(untouched) # returns generated by the market (Long position)
    if average_returns > real_returns:
        beaten = "Success"
    else: 
        beaten = "Failure"
    return([cp, average_returns, real_returns, hit_ratio,beaten,len(g), TimeHorizon])
resultsopen("USD/CAD", "2d")

['USD/CAD', -0.14246082327358406, 0.14246082327358406, 0.0, 'Failure', 1, '2d']

In [13]:
# this part churns out results for long and short strategy
# If this was a day trader of currency pairs following our forecasts,
# he would long a pair or short a pair if signal is positive or negative respectively
# the returns are under the header "IKF Long and Short strategy
# the benchmark is calculated by taking the average of the market performance for each day the trader trades

results1open=[]
for i in listcurr:
    try:
        results1open.append(resultsopen (i , "1d"))# toggle the time horizon to see results
        results1open.append(resultsopen (i , "2d"))
        results1open.append(resultsopen (i , "3d"))
        results1open.append(resultsopen (i , "4d"))
        results1open.append(resultsopen (i , "5d"))
        results1open.append(resultsopen (i , "6d"))
    except:
        continue
dfresultsopen = pd.DataFrame(results1open,columns=["Currency Pair","I Know First Long and Short Strategy", "Benchmark Returns (Long Strategy): ","I Know First Hit Ratio: ","Success?","Appearances in Top 10","Time Horizon"])
dfresultsopen

,Currency Pair,I Know First Long and Short Strategy,Benchmark Returns (Long Strategy):,I Know First Hit Ratio:,Success?,Appearances in Top 10,Time Horizon
0,AUD/GBP,0.127961,0.033695,43.750000,Success,32,1d
1,AUD/GBP,0.109772,0.121441,53.333333,Failure,30,2d
2,AUD/GBP,0.189177,0.259009,60.000000,Failure,30,3d
3,AUD/GBP,0.164642,0.253398,58.620690,Failure,29,4d
4,AUD/GBP,0.045660,0.270729,58.064516,Failure,31,5d
5,AUD/GBP,-0.042465,0.243028,60.000000,Failure,30,6d
6,AUD/JPY,0.147003,-0.054177,100.000000,Success,2,1d
7,AUD/JPY,0.075557,0.075557,100.000000,Failure,1,2d
8,AUD/JPY,0.022739,0.160292,71.428571,Failure,7,3d
9,AUD/JPY,-0.731676,0.871753,66.666667,Failure,3,4d


In [43]:
def addbusdays(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date
def resultstop5open (pair,Hor): 
# this function creates a table and chart for returns and signal
    cp = pair
    TimeHorizon = Hor
    dayreturn = int([char for char in TimeHorizon][0]) + 1
    currpair = []
    # for specific time horizon and specific stock
    for r in range(len(df_top_5)):
        if df_top_5["Currency"][r] == cp and df_top_5["Time Horizon"][r] == TimeHorizon:
            # datetimeobject = datetime.strptime(e["Date"][r],'%d/%m/%Y') # to change from dd/mm/yyyy to a datetime object
            currpair.append([df_top_5["Date"][r], df_top_5["Signal"][r]])# to change to mm/dd/yyyy
    
    g = pd.DataFrame(currpair)
    g.columns = ["Date", "Signal"]
    g.index = g["Date"]
    forcst = []
    retdate = []
    of =[]
    for i in g["Date"]:
        q = datetime.datetime.strptime(i,'%d/%m/%Y') # remove time
        of.append(q.strftime("%m/%d/%Y"))
        forcst.append(q.strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
        retdate.append(addbusdays(q, dayreturn).strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
    g["old format"] = of
    g["Forecast Date"]=forcst
    g["End of return"] = retdate
    dattes = list(finaldf.index)
    newdlist=[]
    for i in range(len(g)):
        if g["Forecast Date"][i] not in dattes:
            newf = (datetime.datetime.strptime(g["old format"][i],"%m/%d/%Y"))-timedelta(days=2)
            newdlist.append(newf.strftime("%m/%d/%Y").replace("2019","19").replace("/0","/").replace("01","1").replace("02","2").replace("03","3").replace("04","4").replace("05","5").replace("06","6").replace("07","7").replace("08","8").replace("09","9"))
        else:
            newdlist.append(g["Forecast Date"][i])
    g["Start of return"]=newdlist
    returns=[]
    for i in range(len(g)):
        try:
            returns.append(((float(combined[cp][g["End of return"][i]]) - dcc[cp][g["Start of return"][i]])/dcc[cp][g["Start of return"][i]])*100)
        except:
            continue
    g["Returns"]= returns
    alif = []
    alll = []
    for i in range(len(g)):
        if (float(g["Signal"][i]) > 0 and g["Returns"][i] > 0) or (float(g["Signal"][i]) < 0 and g["Returns"][i] < 0):
            alll.append(abs(g["Returns"][i]))
        else:
            if g["Returns"][i] < 0:
                alif.append(g["Returns"][i])
            else:
                alif.append(-1*g["Returns"][i])
    alif = [x for x in alif if str(x) != 'nan']
    alll = [x for x in alll if str(x) != 'nan'] 

    if len(alif) > 0:
        average_losses= sum(alif)/len(alif)
    else:
        average_losses= 0
    if len(alll) >0:
        average_gains= sum(alll)/len(alll)
    else:
        average_gains= 0
    average_returns = average_gains + average_losses #Average returns just by following signal per currency pair
    hit_ratio = (len(alll)/len(g))*100
    untouched = []
    for i in g["Returns"]:
        untouched.append(i)
    untouched = [x for x in untouched if str(x) != 'nan']
    real_returns = sum(untouched)/len(untouched) # returns generated by the market (Long position)
    if average_returns > real_returns:
        beaten = "Success"
    else: 
        beaten = "Failure"
    return([cp, average_returns, real_returns, hit_ratio,beaten,len(g), TimeHorizon])
resultstop5open("USD/CAD", "1d")

['USD/CAD', -0.4819640033134996, 0.4819640033134996, 0.0, 'Failure', 1, '1d']

In [15]:
results5topopen=[]
for i in listcurr:
    try:
        results5topopen.append(resultstop5open (i , "1d"))# toggle the time horizon to see results
        results5topopen.append(resultstop5open (i , "2d"))
        results5topopen.append(resultstop5open (i , "3d"))
        results5topopen.append(resultstop5open (i , "4d"))
        results5topopen.append(resultstop5open (i , "5d"))
        results5topopen.append(resultstop5open (i , "6d"))
    except:
        continue
        
dfresults1open = pd.DataFrame(results5topopen,columns=["Currency Pair","I Know First Long and Short Strategy", "Benchmark Returns (Long Strategy): ","I Know First Hit Ratio: ","Success?","Appearances in Top 5", "TimeHorizon"])
dfresults1open

,Currency Pair,I Know First Long and Short Strategy,Benchmark Returns (Long Strategy):,I Know First Hit Ratio:,Success?,Appearances in Top 5,TimeHorizon
0,AUD/GBP,0.171124,0.039466,50.000000,Success,12,1d
1,AUD/GBP,0.295852,-0.091134,44.444444,Success,9,2d
2,AUD/GBP,-0.137376,0.016861,50.000000,Failure,8,3d
3,AUD/GBP,-0.238220,0.081090,40.000000,Failure,5,4d
4,AUD/GBP,-0.344953,0.229770,20.000000,Failure,5,5d
5,AUD/GBP,-0.430026,0.430026,0.000000,Failure,3,6d
6,AUD/USD,0.059331,0.202237,54.545455,Failure,11,1d
7,AUD/USD,0.188342,0.257002,20.000000,Failure,5,2d
8,AUD/USD,0.055593,-0.056495,50.000000,Success,4,3d
9,AUD/USD,0.446955,-0.194454,33.333333,Success,3,4d


In [44]:
export_excel = dfresultsopen.to_excel (r'C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Analysis\IKF Analysis top 10 open.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

print (dfresultsopen)

    Currency Pair  I Know First Long and Short Strategy  \
0         AUD/GBP                              0.127961   
1         AUD/GBP                              0.109772   
2         AUD/GBP                              0.189177   
3         AUD/GBP                              0.164642   
4         AUD/GBP                              0.045660   
5         AUD/GBP                             -0.042465   
6         AUD/JPY                              0.147003   
7         AUD/JPY                              0.075557   
8         AUD/JPY                              0.022739   
9         AUD/JPY                             -0.731676   
10        AUD/JPY                              0.831864   
11        AUD/JPY                              0.968521   
12        AUD/USD                              0.013716   
13        AUD/USD                              0.201758   
14        AUD/USD                              0.057681   
15        AUD/USD                             -0.049937 

In [46]:
export_excel = dfresults1open.to_excel (r'C:\Users\clare\OneDrive\Documents\Internships and work\IKNOWFIRST\IKF WORK\project\Portfolio\Analysis\IKF Analysis top 5 open.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

print (dfresults1open)

    Currency Pair  I Know First Long and Short Strategy  \
0         AUD/GBP                              0.171124   
1         AUD/GBP                              0.295852   
2         AUD/GBP                             -0.137376   
3         AUD/GBP                             -0.238220   
4         AUD/GBP                             -0.344953   
5         AUD/GBP                             -0.430026   
6         AUD/USD                              0.059331   
7         AUD/USD                              0.188342   
8         AUD/USD                              0.055593   
9         AUD/USD                              0.446955   
10        AUD/USD                              0.606322   
11        AUD/USD                              0.601991   
12        CAD/JPY                             -0.268849   
13        CAD/JPY                             -0.305696   
14        CAD/JPY                             -0.136671   
15        CAD/JPY                             -0.237262 